In [1]:
import os
from PIL import Image
import numpy as np
import pickle
from tqdm.autonotebook import tqdm

/tmp/ipykernel_2873718/1379330897.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
classes = [
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "dining_table",
    "dog",
    "horse",
    "motorbike",
    "person",
    "potted_plant",
    "sheep",
    "sofa",
    "train",
    "tv_monitor",
]

In [3]:
def get_image_names_and_one_hots(task_and_ov):
    idxs_path = f"/home/thesis/marx/wilson_gen/WILSON/data/voc/{task_and_ov}/train-{0}.npy"
    idxs = np.load(idxs_path)
    with open("/home/thesis/marx/wilson_gen/WILSON/data/voc/splits/train_aug.txt") as f:
        lines = f.readlines()
        lines = [l.strip().split(" ")[0][1:] for l in lines]
        image_names = np.array(lines)[idxs]
    one_hots = [vec for vec in np.load("/home/thesis/marx/wilson_gen/WILSON/data/voc/voc_1h_labels_train.npy")[idxs]]
    return image_names, one_hots

def get_class(image_name, task):
    # new method, using centrality
    labels = Image.open(f"/home/thesis/marx/wilson_gen/WILSON/data/voc/SegmentationClassAug/{image_name.split('/')[1][:-4]}.png")
    w, h = labels.size
    labels = np.array(labels)
    uniques = np.unique(labels)
    uniques = [unq for unq in uniques if unq > 0 and unq <= int(task.split("-")[0])]
    # print(", ".join([classes[u - 1] for u in uniques]))
    min_dist = 1000000.0
    central_ind = None
    for unq in uniques:
        unq_dists = []
        for i in range(h):
            for j in range(w):
                if labels[i, j] == unq:
                    unq_dists.append(np.sqrt((i - h/2)**2 + (j - w/2)**2))
        if np.mean(unq_dists) < min_dist:
            min_dist = np.mean(unq_dists)
            central_ind = unq
    central_class = classes[central_ind - 1]
    return central_class


In [4]:
# with open("/home/thesis/marx/wilson_gen/WILSON/replay_data_lora/10-10/class_counts.pkl", "rb") as f:
#     class_countsss = pickle.load(f)
#     print(class_countsss)
# with open("/home/thesis/marx/wilson_gen/WILSON/replay_data_lora/10-10/aeroplane/pseudolabels_1h.pkl", "rb") as f:
#     pseduo1hottt = pickle.load(f)
# for k in list(pseduo1hottt.keys())[:2]:
#     print(k, pseduo1hottt[k])
#     print(pseduo1hottt[k].dtype)

In [5]:
TASK = "10-10"
for OVERLAP in [True, False]:
    # prepare data dir
    task_and_ov = f"{TASK}-ov" if OVERLAP else TASK
    print(task_and_ov)
    for cl in classes[:int(TASK.split("-")[0])]:
        os.makedirs(f"/home/thesis/marx/wilson_gen/WILSON/replay_data_voc/{task_and_ov}/{cl}/images", exist_ok=True)
        os.makedirs(f"/home/thesis/marx/wilson_gen/WILSON/replay_data_voc/{task_and_ov}/{cl}/pseudolabels", exist_ok=True)
    # load images and onehot labels
    image_names, one_hots = get_image_names_and_one_hots(task_and_ov)
    print(f"    {len(image_names)} voc images")
    class_counts = {cl: 0 for cl in classes[:int(TASK.split("-")[0])]}
    pseudolabels_1h = {cl: {} for cl in classes[:int(TASK.split("-")[0])]}
    # copy images and pseudolabels into replay folder, create pseudolabel files compatible with replay code
    for i, img_name in tqdm(enumerate(image_names), total=len(image_names)):
        cl = get_class(img_name, TASK)
        # print(cl)
        class_counts[cl] += 1
        img_name = img_name.split("/")[1]
        pseudolabels_1h[cl][f"{img_name[:-4]}.png"] = one_hots[i]
        os.system(f"cp /home/thesis/marx/wilson_gen/WILSON/data/voc/JPEGImages/{img_name} /home/thesis/marx/wilson_gen/WILSON/replay_data_voc/{task_and_ov}/{cl}/images/{img_name}")
        os.system(f"cp /home/thesis/marx/wilson_gen/WILSON/data/voc/SegmentationClassAug/{img_name[:-4]}.png /home/thesis/marx/wilson_gen/WILSON/replay_data_voc/{task_and_ov}/{cl}/pseudolabels/{img_name[:-4]}.png")
    with open(f"/home/thesis/marx/wilson_gen/WILSON/replay_data_voc/{task_and_ov}/class_counts.pkl", "wb") as f:
        pickle.dump(class_counts, f)
    print(f"    {sum(class_counts.values())} gen images")
    for cl in classes[:int(TASK.split("-")[0])]:
        with open(f"/home/thesis/marx/wilson_gen/WILSON/replay_data_voc/{task_and_ov}/{cl}/pseudolabels_1h.pkl", "wb") as f:
            pickle.dump(pseudolabels_1h[cl], f)
        print(f"        {cl}: {len(pseudolabels_1h[cl])}")

10-10-ov
    6139 voc images


  0%|          | 0/6139 [00:00<?, ?it/s]

    6139 gen images
        aeroplane: 580
        bicycle: 420
        bird: 691
        boat: 438
        bottle: 597
        bus: 363
        car: 874
        cat: 994
        chair: 925
        cow: 257
10-10
    3706 voc images


  0%|          | 0/3706 [00:00<?, ?it/s]

    3706 gen images
        aeroplane: 528
        bicycle: 167
        bird: 651
        boat: 291
        bottle: 172
        bus: 205
        car: 466
        cat: 837
        chair: 177
        cow: 212


Looking at the different class counts for overlap and disjoint, we see that especially bicycle, bottle and chair tend to coappear together with future classes. In the case of bottle, this explains the overfitting of the LoRA to bar scenes, as likely 3/4 of the data are that instead of a sole bottle. Looking at the generated images, we see a similar dynamic with chairs in a living room around the dining table. Interestingly, most bicycles still appear alone in the LoRA replay set.